# Can you beat your own agent?

Writing algorithms for RPS is one thing, but playing against your algorithm is another thing! Human vs Machine, Let's find out who is going to rule the world!

![Human vs machine funny picture](https://i.imgur.com/WJ42Vti.png)

**Warning:** DON'T save this notebook as "Save and run all (commit)". It will run forever (because it deploys a FLASK service). "Quick save" is the only way to save copies of this notebook.

In [ ]:
!pip install flask-ngrok
!mkdir templates
!mkdir static

In [ ]:
%%writefile static/style.css
*{
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}
body{
    min-height: 100vh;
    display: flex;
    flex-direction: column;
    align-items: center;
}
h1{
    font-family: sans-serif;
    font-weight: 800;
    margin: 15px 0;
}
.container{
    display: flex;
}
.container .section{
    width: 350px;
    margin: 20px 50px;
}
.section .info{
    background: rgb(10, 10, 10);
    display: flex;
    align-items: center;
    justify-content: center;
    height: 50px;
}
.info span , .info h3{
    font-size: 25px;
    font-weight: bold;
    margin: 0px 10px;
    font-family: sans-serif;
    color: white;
}
.show{
    display: flex;
    align-items: center;
    justify-content: center;
    background : rgb(41, 40, 40);
    height: 220px;
}
.show i{
    font-size: 100px;
    color: orange;
}
h2{
    font-family: sans-serif;
    font-weight: bold;
    padding: 5px;
}
.selection{
    display: flex;
    align-items: center;
    justify-content: center;
    width: 350px;
    height: 100px;
}
.selection button{
    margin: 0 20px;
    padding: 25px;
    border-radius: 50%;
    border: none;
    outline: none;
    cursor: pointer;
    background : rgb(41, 40, 40);
    font-size: 33px;
    color: orange;
}

In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>R-P-S Game</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
    <script src="https://kit.fontawesome.com/e391ce7786.js" crossorigin="anonymous"></script>
</head>
<body>
    <h1>Rock Paper Scissors</h1>
    <h2 id="demo">Try your luck !</h2>
<!--  The Container  -->
    <div class="container">
<!--   The first section     -->
        <div class="section">
            <div class="info">
                <h3>You :</h3><span id="playerScore">0</span>
            </div>
            <div class="show">
                <i class="fas fa-hand-rock" id="my_last_action"></i>
            </div>
        </div>
<!--   The second section     -->
        <div class="section">
            <div class="info">
                <h3>Computer :</h3><span id="computerScore">0</span>
            </div>
            <div class="show computer">
                <i class="fas fa-hand-scissors" id="opp_last_action"></i>
            </div>
        </div>
    </div>
    <h2 style="color: rgb(233, 33, 19);" id="demo2">Choose One !</h2>
<!-- This selection div contains the buttons   -->
    <div class="selection">
        <button name="action" class="fas fa-hand-rock" value="rock" type="submit"></button>
        <button name="action" class="fas fa-hand-paper" value="paper" type="submit"></button>
        <button name="action" class="fas fa-hand-scissors" value="scissors" type="submit"></button>
    </div>
    
    <script src="//ajax.googleapis.com/ajax/libs/jquery/1.9.1/jquery.min.js"></script>
    <script type=text/javascript>
        $(function() {
          $('button[name=action]').bind('click', function() {
            var action_url = "/action/" + $(this).val() + "/";
            $.getJSON(action_url,
                function(data) {
              $("#my_last_action").removeClass().addClass("fas").addClass(data["action_class"]);
              $("#opp_last_action").removeClass().addClass("fas").addClass(data["agent_action_class"]);
              $("#playerScore").html(data["our_reward"]);
              $("#computerScore").html(data["agent_reward"]);
            });
            return false;
          });
        });
    </script>
</body>
</html>

In [ ]:
%%writefile templates/win.html
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>R-P-S Game</title>
</head>
<body>
<div class="container">
<img src="https://i.imgur.com/g2ORIlE.png" width="500px">
</div>
</body>
</html>

# Attach your submission to this notebook ("Add Data" on the right side)

Just attach your notebook with your `submission.py` or `main.tar.gz` here. And set the path to your agent and number of game steps below.

In [ ]:
AGENT_PATH = '../input/rps-dojo/white_belt/reactionary.py'  # path to agent file (*.py or *.tar.gz)
GAME_STEPS = 100  # Number of steps to play

# Deploying a Flask service using ngrok

If you're running this notebook on Kaggle, make sure to run this cell **in interactive session**. If you're running it locally, you can remove the `run_with_ngrok(app)` line. This cell's output will look like this:

![Cell output example](https://i.imgur.com/aTWmaY2.png)

Just click on the link `something.ngrok.io` and you will see the following interface:

![App interface](https://i.imgur.com/cxQm5GS.png)

As you can see, I'm losing even to a simple `reactionary` agent. How about you? 😁😁😁

In [ ]:
import os
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, jsonify
from kaggle_environments import make, Agent

# Start a FLASK app
TEMPLATE_DIR = os.path.abspath('templates')
STATIC_DIR = os.path.abspath('static')
app = Flask(__name__, template_folder=TEMPLATE_DIR, static_folder=STATIC_DIR)

# Start RPS environment
global env
global agent
env = make("rps", configuration={"episodeSteps": GAME_STEPS})
agent = Agent(AGENT_PATH, env)

run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/")
def home():
    return render_template('index.html', last_action='rock')

@app.route("/win/")
def you_won():
    return render_template('win.html')

@app.route("/action/<action>/")
def move_forward(action):
    # executing one RPS environment step
    global env
    global agent
    our_action = {'rock': 0, 'paper': 1, 'scissors': 2}[action]
    agent_action, logs = agent.act(env.steps[-1][0].observation)
    env.step([agent_action, our_action])
    
    # If the game is over (either ended or the agent threw an exception)...
    
    # Render the JSON response
    opponent_action = ['rock', 'paper', 'scissors'][agent_action]
    last_action_class = f'fa-hand-{action}'
    last_agent_action_class = f'fa-hand-{opponent_action}'
    response = jsonify({
        'action_class': last_action_class,
        'agent_action_class': last_agent_action_class,
        'our_reward': int(env.state[1].reward),
        'agent_reward': int(env.state[0].reward),
    })
    return response
  
app.run()